House features (square footage, bedrooms, bathrooms, age) to predict price.

Build a neural network using PyTorch with the following architecture:
* Input layer: 4 features
* Hidden layer 1: 16 neurons, ReLU activation
* Hidden layer 2: 8 neurons, ReLU activation
* Output layer: 1 neuron (regression, no activation)


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# x_train = np.random.rand(500, 4)
# y_train = np.random.rand(500, 1)*10000

x = torch.rand(500, 4)
y = x.sum(dim=1, keepdim=True)*250

# x_train_torch = torch.tensor(x_train)
# y_train_torch = torch.tensor(y_train)

# x_train = x_train.to('mps') #use cude if running on gpu i have mac so using the mps
# y_train = y_train.to('mps') # we could do y_train = torch.rand(500, 4, device='mps') same thing

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2) 

x_train = x_train.to('mps')
x_test = x_test.to('mps')
y_train = y_train.to('mps')
y_test = y_test.to('mps')

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_dataset = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = DataLoader(test_dataset, batch_size=64, shuffle=True)


class HouseModel(nn.Module):
    def __init__(self):
        super(HouseModel, self).__init__()
        self.f1 = nn.Linear(4, 16)
        self.f2 = nn.Linear(16, 8)
        self.output = nn.Linear(8, 1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.f1(x))
        x = self.relu(self.f2(x))
        x = self.output(x)
        return x

model = HouseModel().to('mps')

epochs = 50
crit = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=0.02)

for i in range(epochs):
    model.train()
    for xi, yi in train_dataset:
        optim.zero_grad()
        predection = model(xi)
        loss = crit(predection, yi)
        loss.backward()
        optim.step()

model.eval()
with torch.no_grad():
    test_loss = 0
    for x, y in test_dataset:
        pred = model(x)
        loss = crit(pred, y)
        test_loss += loss.item()
    avg_test_loss = test_loss / len(test_dataset)
    print(f"Average Test Loss: {avg_test_loss:.2f}")


Average Test Loss: 96.23
